In [1]:
path_to_file = 'martineden.txt'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 784031 characters


In [2]:
#Look at first 1000 Characters
print(text[:1000])

﻿Martin Eden

by Jack London

“Let me live out my years in heat of blood!
    Let me lie drunken with the dreamer’s wine!
Let me not see this soul-house built of mud
    Go toppling to the dust a vacant shrine!”




CHAPTER I.


The one opened the door with a latch-key and went in, followed by a
young fellow who awkwardly removed his cap. He wore rough clothes that
smacked of the sea, and he was manifestly out of place in the spacious
hall in which he found himself. He did not know what to do with his
cap, and was stuffing it into his coat pocket when the other took it
from him. The act was done quietly and naturally, and the awkward young
fellow appreciated it. “He understands,” was his thought. “He’ll see me
through all right.”

He walked at the other’s heels with a swing to his shoulders, and his
legs spread unwittingly, as if the level floors were tilting up and
sinking down to the heave and lunge of the sea. The wide rooms seemed
too narrow for his rolli


In [3]:
# List all unique characters in this dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$%&(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyzÆàäéôö—‘’“”﻿
90


In [4]:
# Map from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] #Take a string, output list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #Take a list of integers, output a string

print(encode("Hello World"))
print(decode(encode("Hello World")))

[32, 56, 63, 63, 66, 2, 47, 66, 69, 63, 55]
Hello World


In [5]:
#Now we encode the entire dataset and store it in a torch.Tensor using PyTorch
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) #These 1000 characters will be displayed how the GPT model views text

torch.Size([784031]) torch.int64
tensor([89, 37, 52, 69, 71, 60, 65,  2, 29, 55, 56, 65,  1,  0,  1,  0, 53, 76,
         2, 34, 52, 54, 62,  2, 36, 66, 65, 55, 66, 65,  1,  0,  1,  0, 87, 36,
        56, 71,  2, 64, 56,  2, 63, 60, 73, 56,  2, 66, 72, 71,  2, 64, 76,  2,
        76, 56, 52, 69, 70,  2, 60, 65,  2, 59, 56, 52, 71,  2, 66, 57,  2, 53,
        63, 66, 66, 55,  3,  1,  0,  2,  2,  2,  2, 36, 56, 71,  2, 64, 56,  2,
        63, 60, 56,  2, 55, 69, 72, 65, 62, 56, 65,  2, 74, 60, 71, 59,  2, 71,
        59, 56,  2, 55, 69, 56, 52, 64, 56, 69, 86, 70,  2, 74, 60, 65, 56,  3,
         1,  0, 36, 56, 71,  2, 64, 56,  2, 65, 66, 71,  2, 70, 56, 56,  2, 71,
        59, 60, 70,  2, 70, 66, 72, 63, 10, 59, 66, 72, 70, 56,  2, 53, 72, 60,
        63, 71,  2, 66, 57,  2, 64, 72, 55,  1,  0,  2,  2,  2,  2, 31, 66,  2,
        71, 66, 67, 67, 63, 60, 65, 58,  2, 71, 66,  2, 71, 59, 56,  2, 55, 72,
        70, 71,  2, 52,  2, 73, 52, 54, 52, 65, 71,  2, 70, 59, 69, 60, 65, 56,
       

In [6]:
#Split data into Train and Validation sets. First 90% will be train and the rest will be val.
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8
train_data[:block_size+1]

tensor([89, 37, 52, 69, 71, 60, 65,  2, 29])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([89]) the target: 37
when input is tensor([89, 37]) the target: 52
when input is tensor([89, 37, 52]) the target: 69
when input is tensor([89, 37, 52, 69]) the target: 71
when input is tensor([89, 37, 52, 69, 71]) the target: 60
when input is tensor([89, 37, 52, 69, 71, 60]) the target: 65
when input is tensor([89, 37, 52, 69, 71, 60, 65]) the target: 2
when input is tensor([89, 37, 52, 69, 71, 60, 65,  2]) the target: 29


In [9]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will we process in parallel?
block_size = 8 # What is the maximum context length for predictions?

def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[60, 65, 58,  2, 59, 60, 70,  2],
        [69, 66, 66, 64, 70, 11,  2, 27],
        [66,  2, 60, 65, 70, 72, 63, 71],
        [60, 58, 59, 71, 60, 65, 58,  1]])
targets:
torch.Size([4, 8])
tensor([[65, 58,  2, 59, 60, 70,  2, 69],
        [66, 66, 64, 70, 11,  2, 27, 66],
        [ 2, 60, 65, 70, 72, 63, 71,  2],
        [58, 59, 71, 60, 65, 58,  1,  0]])
----
when input is [60] the target: 65
when input is [60, 65] the target: 58
when input is [60, 65, 58] the target: 2
when input is [60, 65, 58, 2] the target: 59
when input is [60, 65, 58, 2, 59] the target: 60
when input is [60, 65, 58, 2, 59, 60] the target: 70
when input is [60, 65, 58, 2, 59, 60, 70] the target: 2
when input is [60, 65, 58, 2, 59, 60, 70, 2] the target: 69
when input is [69] the target: 66
when input is [69, 66] the target: 66
when input is [69, 66, 66] the target: 64
when input is [69, 66, 66, 64] the target: 70
when input is [69, 66, 66, 64, 70] the target: 11
when input is [6

In [10]:
print(xb) #Our input to the transformer

tensor([[60, 65, 58,  2, 59, 60, 70,  2],
        [69, 66, 66, 64, 70, 11,  2, 27],
        [66,  2, 60, 65, 70, 72, 63, 71],
        [60, 58, 59, 71, 60, 65, 58,  1]])


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Get predictions
            logits, loss = self(idx)
            # Focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 90])
tensor(4.7894, grad_fn=<NllLossBackward0>)

cz9j7hU0ô’Dät,zW_ÆBE37VX“tmD5gcC;
9o5äU àL)0R5,9
ätmk“8aäyM.:


In [12]:
#Create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4262759685516357


In [18]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


coro!ZHondrt. Thomuthewherorasshamory d ag blloumajebisthacrgeh s thd, titht shoto w.
ascsune ajug Nof t t balache, setin. fr Omandig pes
caco r bythed pt he-scoiled bre.8!” f—ghik Seaupran-gulded thent allame titomene eawamupaneve HAnto blalall t.’se wherad,’s waure thath r Ratweeresis ontrabe th
cay tutcade ied Pranin, gesthopess’As th whippronsed,”:

guaknghe santichod r ff mme an, hesor
cot flerk, L2DSobed

tu’thal.
BMasut
kertrmobe alithe7Zxpuk. ofime owr “Fme ate grofe Ove grinve


In [19]:
# Toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [20]:
# Consider the following example.
torch.manual_seed(1337)
B,T,C = 4,8,2 #Batch, Time, Channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [21]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [22]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [23]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [24]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [25]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [26]:
# version 4: Self-Attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # Batch, Time, Channels
x = torch.randn(B,T,C)

# Let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [27]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [28]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [29]:
k.var()

tensor(1.0449)

In [30]:
q.var()

tensor(1.0700)

In [31]:
wei.var()

tensor(1.0918)

In [32]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [33]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [34]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # Calculate the forward pass
    xmean = x.mean(1, keepdim=True) # Batch mean
    xvar = x.var(1, keepdim=True) # Batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # Normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # Batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [35]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [36]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [39]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 16 # How many independent sequences will we process in parallel?
block_size = 32 # What is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)


path_to_file = 'martineden.txt'
# Read, then decode for py2 compat.
with open(path_to_file, 'r', encoding='utf-8') as f:
    text = f.read()


# Here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# Create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: Take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # Decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # First 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# Data loading
def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ One head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # Compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # Perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ A simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


# Super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # Get the predictions
            logits, loss = self(idx_cond)
            # Focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# Print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # Every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.212825 M parameters
step 0: train loss 4.6685, val loss 4.6640
step 100: train loss 2.6222, val loss 2.6334
step 200: train loss 2.4903, val loss 2.4937
step 300: train loss 2.4185, val loss 2.4311
step 400: train loss 2.3298, val loss 2.3299
step 500: train loss 2.2664, val loss 2.2707
step 600: train loss 2.2135, val loss 2.2121
step 700: train loss 2.1512, val loss 2.1640
step 800: train loss 2.1052, val loss 2.1194
step 900: train loss 2.0690, val loss 2.0710
step 1000: train loss 2.0470, val loss 2.0431
step 1100: train loss 2.0175, val loss 2.0256
step 1200: train loss 1.9775, val loss 2.0014
step 1300: train loss 1.9608, val loss 1.9743
step 1400: train loss 1.9377, val loss 1.9424
step 1500: train loss 1.9204, val loss 1.9345
step 1600: train loss 1.9024, val loss 1.8951
step 1700: train loss 1.8795, val loss 1.8962
step 1800: train loss 1.8596, val loss 1.8798
step 1900: train loss 1.8633, val loss 1.8687
step 2000: train loss 1.8492, val loss 1.8703
step 2100: train loss 1.